In [1]:
import cv2
from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

pdf_path = '../pdf_file/1/bill.pdf'
images = convert_from_path(pdf_path, grayscale=True, jpegopt=True)
for img in images:
    text = pytesseract.image_to_string(img, lang='vie')[:-1]
    print(text)


[ WARN:0@0.181] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../pdf_file/1/bill.jpg'): can't open/read file: check file path/integrity


TypeError: Unsupported image object

EXTRACT AND MERGE TO PARAGRAPH

In [4]:
import cv2
import pytesseract
from pytesseract import Output

img_path = '../test_img/1/vbg_27.jpg'
img = cv2.imread(img_path)

##########################################################################
class Paragraph:

    def __init__(self, img, para_info):
        self._para_info = para_info
        self._img = img
        self._lines_coords = None
        self._lines_ct = None
        self._paragr_coord = None
        
    @property
    def lines_ct(self):
        if self._lines_ct is None:
            self._lines_ct = self._para_info[0]
        return self._lines_ct

    @property
    def lines_coords(self):
        if self._lines_coords is None:
            self._lines_coords = self._para_info[1]
        return self._lines_coords  

    @property
    def paragraph_content(self):
        paragraph = ''
        for line in self.lines_ct:
            for word in line:
                paragraph += word + ' '
        return paragraph
    
    @property
    def paragraph_coord(self):
        if self._paragr_coord is None:
            x1 = min([f[0] for f in self.lines_coords])
            y1 = self.lines_coords[0][1]
            x2 = max([f[2] for f in self.lines_coords])
            y2 = self.lines_coords[-1][3]
            self._paragr_coord = [x1, y1, x2, y2]
        return self._paragr_coord
    
    @property
    def paragraph_img_crop(self):
        x1, y1, x2, y2 = self.paragraph_coord
        paragr_img = self._img[y1:y2, x1:x2]
        return paragr_img
########################################################################

#######################################################
class Line:

    def __init__(self, img, line_coord, line_ct):
        self._img = img
        self._line_coord = line_coord
        self._line_ct = line_ct
    
    @property
    def line_content(self):
        line = ''
        for word in self._line_ct:
            line += word + ' '
        return line
    
    @property
    def line_img_crop(self):
        x1, y1, x2, y2 = self._line_coord
        crop = self._img[y1:y2, x1:x2]
        return crop
#######################################################

text_info = pytesseract.image_to_data(
    img,
    lang='vie',
    output_type=Output.DICT
)
remove_ids = []
for id, text in enumerate(text_info['text']):
    if text  == "" or text == " ":
        remove_ids.append(id)
for id, conf in enumerate(text_info['conf']):
    if conf == '-1' or conf <= 40:
        remove_ids.append(id)
remove_ids = sorted(list(set(remove_ids)))
for key in text_info.keys():
    for id, val in enumerate(remove_ids):
        del text_info[key][val - id]
extract_ids = {}
for ch_id in range(text_info['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_info['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_info['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_info['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
# print(extract_ids)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_info['left'][begin_id], text_info['top'][begin_id], text_info['width'][begin_id], text_info['height'][begin_id]
            e_x, e_y, e_w, e_h = text_info['left'][end_id], text_info['top'][end_id], text_info['width'][end_id], text_info['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_info['text'][word_idx])
            pr_lines_ct.append(line_ct)
        # print(pr_lines_ct)
        # print('.......................')
        need_to_remove = len(pr_lines_ct[0]) == 1 and len(pr_lines_ct[0][0]) <= 3 and pr_lines_ct[0][0].isdecimal()
        if need_to_remove: 
            pass
        else: 
            paragraphs_infor.append([pr_lines_ct, pr_lines_coords])

for paragr in paragraphs_infor:
    print(paragr[0])

[['PHẢN', 'GHI', 'CỦA', 'KHO', 'BẠC', 'NHÀ', 'NƯỚC']]
[['Ngày', 'nhận', 'giấy', 'đề', 'nghị', 'thanh', 'toán']]
[['[Kho', 'bạc', 'Nhà', 'nước', 'châễ', 'nhận', 'Thanh', 'toán', '-', 'Theo', 'nội', 'dung']]
[['Kho', 'bạc', 'Nhà', 'nước', 'chấp', 'thuận', 'thanh', 'toán', 'như', 'sau']]
[['Nội', 'dun', 'Tông', 'sô', 'Vôn', 'trong', 'nước', 'ˆ', '_', 'Vôn', 'nước', 'ngoài'], ['Sô', 'vôn', 'châp', 'nhân', '—']]
[['Mục,', 'tiêu', 'mục', '-', '-'], ['Mục,', 'tiểu', 'mục'], ['tiêu', 'mục', '-'], ['Mục,', 'tiêu', 'mục', ':', '_']]
[['Các']]
[['năm']]
[['trước']]
[['Năm']]
[['nay']]
[['_']]
[['-']]
[['Thuê']]
[['giá']]
[['trị']]
[['gia'], ['tăng']]
[['Sô']]
[['đơn']]
[['vị']]
[['thụ', 'hưởng', '_'], ['Băng', 'chữ', ':', '-', ':']]
[['Từ', 'chội'], ['Lý', 'do:']]
[['Ghi', 'chú:', '.......']]
[['CHUYÊN', 'VIÊN', 'KIỂM', 'SOÁT', 'CHUGIAO', 'DỊCH', 'VIÊN', 'TRƯỞNG', 'PHÒNG', 'GIÁM', 'ĐÓC', 'KHO', 'BẠC', 'NHÀ', 'NƯỚC'], ['(Kỷ,', 'ghi', 'rõ', 'họ', 'tên)', '(Ký,', 'ghi', 'rõ', 'họ', 'tên)', '(Ký,', '

EXTRACT DATA DIALOGUE

In [31]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../experiment/capture/Capture.JPG'
img = cv2.imread(path)

def merge_content(lines_ct: list):
    content = ""
    for line in lines_ct:
        for word in line:
            content += word + ' '
    return content

text_infor = pytesseract.image_to_data(Image.fromarray(img), lang='vie', output_type=Output.DICT)
remove_ids = []
for id, text in enumerate(text_infor['text']):
    if text  == "" or text == " ":
        remove_ids.append(id)
for key in text_infor.keys():
    for id, val in enumerate(remove_ids):
        del text_infor[key][val - id]
        
extract_ids = {}
for ch_id in range(text_infor['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_infor['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_infor['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_infor['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_infor['left'][begin_id], text_infor['top'][begin_id], text_infor['width'][begin_id], text_infor['height'][begin_id]
            e_x, e_y, e_w, e_h = text_infor['left'][end_id], text_infor['top'][end_id], text_infor['width'][end_id], text_infor['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_infor['text'][word_idx])
            pr_lines_ct.append(line_ct)
        paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
    
for para in paragraphs_infor:
    print(merge_content(para[0]))

A: Are you here on vacation? Anh đền đây du lịch à? 
B: No, Pm not. Im here working. Không. Tôi đi công tác. 
: Have you been to VietNam before? Anh đã từng đền Việt Nam chưa? 
B: No. lts my first time to com here. Chưa. Đây là làn đầu tiên tôi đền đây. 
A: How long will you stay here? 
Anh sẽ ở lại Việt Nam trong bao lâu? B: A week. 
Tôi ở lại đây 1 tuần 
A : Can you speak Vietnamese? Anh có biết tiếng Việt không? 
B:Oh, alitle Ô, chỉ 1 chút thôi. 


In [1]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../experiment/capture/Capture.JPG'
img = cv2.imread(path)

def merge_content(lines_ct: list):
    content = ""
    for line in lines_ct:
        for word in line:
            content += word + ' '
    return content

text_infor = pytesseract.image_to_data(img, lang='vie', output_type=Output.DICT)
remove_ids = []
for id, text in enumerate(text_infor['text']):
    if text  == "" or text == " ":
        remove_ids.append(id)
for key in text_infor.keys():
    for id, val in enumerate(remove_ids):
        del text_infor[key][val - id]
        
extract_ids = {}
for ch_id in range(text_infor['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_infor['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_infor['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_infor['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_infor['left'][begin_id], text_infor['top'][begin_id], text_infor['width'][begin_id], text_infor['height'][begin_id]
            e_x, e_y, e_w, e_h = text_infor['left'][end_id], text_infor['top'][end_id], text_infor['width'][end_id], text_infor['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_infor['text'][word_idx])
            pr_lines_ct.append(line_ct)
        paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
    
for para in paragraphs_infor:
    print(merge_content(para[0]))

For how long have you studied English? Anh đã học Hếng Anh bao lâu rồi: 
Let 
me see, At least ten years, I think. Để xem nào. Tôi nghĩ ít nhất 10 năm rồi. 
$o you need no more grammar Iesson. Vậy anh không cần các bài ngữ pháp nữu. 
Why2 How could I know rules of talking English then? 
Sao uậy? Thế thì làm sao lôi biết các qui tắc nói Hếng Anh được? 
Practically, we talk English, but most of students here talk: about English rules. 
Thực tế, chúng lôi nói tiếng Anh, nhưng hầu hết các học sinh ở đây nói bề các qui hắc Hếng Anh. 
How should we do then? Thể thì chúng tôi nên làm thế nào? 
You have to learn how to talk. And you learn to talk only by talking 
Anh phải học cách nói chuyện. Và anh học nói chuyện chỉ bằng cách nói chuyện. 


In [3]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../test_img/2/vbg_0.jpg'
img = cv2.imread(path)

def merge_content(lines_ct: list):
    content = ""
    for line in lines_ct:
        for word in line:
            content += word + ' '
    return content

text_infor = pytesseract.image_to_data(Image.fromarray(img), lang='vie', output_type=Output.DICT)
remove_ids = []
for id, text in enumerate(text_infor['text']):
    if text  == "" or text == " ":
        remove_ids.append(id)
for key in text_infor.keys():
    for id, val in enumerate(remove_ids):
        del text_infor[key][val - id]
        
extract_ids = {}
for ch_id in range(text_infor['block_num'].__len__()):
    if not bool(extract_ids):
        extract_ids = {1: {1: {1: [ch_id]}}}
    else:
        max_block_num = (list(extract_ids.keys()))[-1]
        max_par_num = (list(extract_ids[max_block_num].keys()))[-1]
        max_line_num = (list(extract_ids[max_block_num][max_par_num].keys()))[-1]
        if text_infor['block_num'][ch_id] != max_block_num:
            extract_ids[max_block_num + 1] = {1: {1: [ch_id]}}
        else: 
            if text_infor['par_num'][ch_id] != max_par_num:
                extract_ids[max_block_num][max_par_num + 1] = {1: [ch_id]}
            else:
                if text_infor['line_num'][ch_id] != max_line_num:
                    extract_ids[max_block_num][max_par_num][max_line_num + 1] = [ch_id]
                else:
                    extract_ids[max_block_num][max_par_num][max_line_num].append(ch_id)
paragraphs_infor = []
for bl in extract_ids.keys():
    for par in extract_ids[bl].keys():
        pr_lines_ct = []
        pr_lines_coords = []
        for line in extract_ids[bl][par].keys():
            begin_id = extract_ids[bl][par][line][0]
            end_id = extract_ids[bl][par][line][-1]
            b_x, b_y, b_w, b_h = text_infor['left'][begin_id], text_infor['top'][begin_id], text_infor['width'][begin_id], text_infor['height'][begin_id]
            e_x, e_y, e_w, e_h = text_infor['left'][end_id], text_infor['top'][end_id], text_infor['width'][end_id], text_infor['height'][end_id]
            x1, y1, x2, y2 = b_x, min(b_y, e_y), e_x + e_w, max(b_y + b_h, e_y, e_h)
            pr_lines_coords.append([x1, y1, x2, y2])
            line_ct = []
            for word_idx in extract_ids[bl][par][line]:
                line_ct.append(text_infor['text'][word_idx])
            pr_lines_ct.append(line_ct)
        paragraphs_infor.append([pr_lines_ct, pr_lines_coords])
    
for para in paragraphs_infor:
    print(merge_content(para[0]))

' Luật số: 56/2020/QH14 
Ký bởi: Cổng Thông tin điện tử Chính phủ Email: thongtinchinhphu@chinhphu.vn Cơ quan: Văn phòng Chính phủ 
` Thời gian ký: 08.07.2020 15:51:37 +07:00 
QUÓC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc 
SỬA ĐỎI, BỎ SUNG MỘT SỐ ĐIỀU CỦA LUẬT GIÁM ĐỊNH TƯ PHÁP 
Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam; 
Quốc hội ban hành Luật sửa đổi, bồ sung một số điều của Luật Giám định tư pháp số 13/2012/QH13 đã được sửa đổi, bồ sung một số điều theo Luật số 35/2018/QH14. 
Điều 1. Sửa đổi, bổ sung một số điều của ILuât Giám định tư pháp 
1. Sửa đox, bồ sung các khoản 1, 2 và 3 Điều 2như sau: 
“1. Giám định tư phap là việc người giám định tư pháp sử dụng k1en thức, phương tiện, phương pháp khoa học, kỹ thuật, ngh1ep vụ đề kết luận về chuyên môn nhung vấn đề có liên quan đến hoạt động khởi tố, điều tra, truy tố, xét xử và 
- thi hành án hình sự, giải quyet vụ việc dan sự, vụ án hành chính theo "trung cau : của cơ quan có tham quyền tiền hành

In [7]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output


path = '../experiment/capture/Capture.JPG'
img = cv2.imread(path)
text_infor = pytesseract.image_to_string(Image.fromarray(img), lang='vie')[:-1].split('\n')
text_infor

['Amy',
 '',
 'Michael',
 '',
 'Amy',
 '',
 'Michael',
 'Amy',
 'Michael',
 '',
 'Amy',
 '',
 'Chào Michael,',
 '',
 'Hi Michael.',
 '',
 'Chào Amy. Có chuyện gì vậy?',
 '',
 'H¡ Amy. Whats up?',
 '',
 'TTôi đang tìm phi trường. Bạn có thể nói cho',
 'tôi biết làm cách nào đến đó được không?.',
 'Im looking for the airport. Can you tell me',
 'how to getthere?',
 '',
 'Không, tất tiếc. Tôi không biết.',
 '',
 'No, sony. I don know.',
 '',
 'Tôi nghĩ tôi có thể đ xe điện ngầm để tới phí',
 'trường. Bạn có biết đuờng xe điện ngầm ở.',
 'đâu không?.',
 '',
 'l think l can take the subway to the airport.',
 'Do you know where the subway is?',
 '',
 'Chắc rổi. Nó ở đẳng kia.',
 '',
 'Sure,ifs over there.',
 '',
 "'Ở đâu? Tôi không thấy nó.",
 '',
 'Where? I dontsee it.',
 '',
 'Phía bên kia đường.',
 '',
 'Across the street.',
 '',
 'Ô, bây giờ tôi thấy rồi. Cám ơn.',
 '',
 'Oh, I see it now. Thanks.',
 '',
 ' ',
 '']